In [1]:
import numpy as np
import pandas as pd
import matplotlib as matplot
import nltk

# Question 1

In [2]:
en_df = pd.read_csv('data/CONcreTEXT_trial_EN.tsv', sep='\t') # load data files
it_df = pd.read_csv('data/CONcreTEXT_trial_IT.tsv', sep='\t')

In [3]:
en_df['LANGUAGE'] = ['ENGLISH'] * en_df.shape[0]
it_df['LANGUAGE'] = ['ITALIAN'] * it_df.shape[0]

In [4]:
df = pd.concat([en_df, it_df])
df = df.reset_index(0)
df

,index,TARGET,POS,INDEX,TEXT,MEAN,LANGUAGE
0,0,achievement,N,3,"Bring up academic achievements , awards , and ...",3.06,ENGLISH
1,1,achievement,N,9,"Please list people you have helped , your pers...",3.03,ENGLISH
2,2,activate,V,1,Add activated carbon straight to your vodka .,3.83,ENGLISH
3,3,activate,V,15,"Place sensors around your garden , and when a ...",5.51,ENGLISH
4,4,adventure,N,9,Look for a partner that shares your level of a...,2.03,ENGLISH
...,...,...,...,...,...,...,...
195,95,verità,N,8,"In un modo o nell' altro , la verità viene sem...",2.53,ITALIAN
196,96,viaggio,N,2,Organizza dei viaggi nel fine settimana quando...,5.03,ITALIAN
197,97,viaggio,N,6,Pesa le tue valigie prima del viaggio per evit...,4.84,ITALIAN
198,98,vista,N,6,è molto importante non perdere di vista la pro...,2.22,ITALIAN


# Question 2

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

In [10]:
train = count_vectorizer.fit_transform(df['TEXT'])

In [16]:
train[0]

<1x1330 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>